In [1]:
import tensorflow as tf
import numpy as np

In [2]:
from src.data import Data
from src.plothelp import *
import time

### Load and pre-process the dataset:


In [3]:
data = Data("img")
data.load_pickle("100x100.pickle ")
X_train, X_test, y_train, y_test = data.train_test_split()

In [4]:
X_trains = np.asarray(X_train).reshape((len(X_train), -1))
X_tests = np.asarray(X_test).reshape((len(X_test), -1))
y_trains = np.asarray(y_train)
y_tests = np.asarray(y_test)
print(X_trains.shape, X_tests.shape, y_trains.shape, y_tests.shape)

(4698, 30000) (522, 30000) (4698,) (522,)


In [5]:
y_oh, y_test_oh= data.one_hot(y_trains, y_tests)

In [6]:
# Split y_train into 54 batches containing 87 img
y_batch = np.array_split(y_oh, 54 )
y_batch = np.asarray(y_batch)
print(y_batch.shape)

(54, 87, 4)


In [7]:
# Split X_test into 54 batches of 87 img
X_train_batch = np.array_split(X_trains, 54 )
X_train_batch = np.asarray(X_train_batch)
print(X_train_batch.shape)
print(X_train_batch.shape)

(54, 87, 30000)
(54, 87, 30000)


In [8]:
learning_rate = 0.0001
batch_size = 87
n_epochs = 30
dropout = 0.5

In [9]:
X = tf.placeholder(tf.float32, [None, 30000])
Y = tf.placeholder(tf.float32, [None, 4])

In [10]:
#Number of neurons in layers
in_n = 30000 #input layer
l2 = 256  #layer 2
l3 = 256  #layer 3
final = 4 #final layer


w1 = tf.Variable(tf.random_normal(shape=[in_n, l2], stddev=0.01))
b1 = tf.Variable(tf.random_normal([l2]))
w2 = tf.Variable(tf.truncated_normal([l2, l3], stddev=0.01))
b2 = tf.Variable(tf.random_normal([l3]))
w3 = tf.Variable(tf.truncated_normal([l3, final], stddev=0.01))
b3 = tf.Variable(tf.random_normal([final]))

In [11]:
#The network model
y1 = tf.nn.relu(tf.matmul(X, w1) + b1)
#y1 = tf.nn.dropout(y1,dropout)
y2 = tf.nn.relu(tf.matmul(y1, w2) + b2)
#y2 = tf.nn.dropout(y2,dropout)
logits = tf.matmul(y2, w3) + b3


In [12]:
entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y)
loss = tf.reduce_mean(entropy)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

acc = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
acc = tf.reduce_mean(tf.cast(acc, tf.float32))
  
saver = tf.train.Saver()

In [13]:
init = tf.global_variables_initializer()

In [14]:
with tf.Session() as sess:
  sess.run(init)
  batch_img = 54
  for i in range(n_epochs):  # train the model n_epochs times
        avg_acc = 0.
        avg_cost = 0.
        
        for j in range(batch_img):
            X_batch = X_train_batch[j,:]
            Y_batch = y_batch[j,:]
            _,c, a =sess.run([optimizer, loss, acc], feed_dict={X: X_batch, Y: Y_batch})
            avg_acc += a/batch_img
            avg_cost += c / batch_img
            
        print("Epoch:", '%03d' % (i+1), "cost=", avg_cost, "accuracy=",avg_acc)
            
            
  save_path = saver.save(sess, "./models/3layer-lr=0.0001.ckpt")
  print("Model saved in file: %s" % save_path)

Epoch: 001 cost= 2.55071395194 accuracy= 0.357598980268
Epoch: 002 cost= 1.16828697478 accuracy= 0.468922946188
Epoch: 003 cost= 1.1157141372 accuracy= 0.509578542577
Epoch: 004 cost= 1.07089120039 accuracy= 0.536824177813
Epoch: 005 cost= 1.0367806598 accuracy= 0.563005535139
Epoch: 006 cost= 1.0016512926 accuracy= 0.590251167615
Epoch: 007 cost= 0.99519857102 accuracy= 0.593656870502
Epoch: 008 cost= 0.964340439549 accuracy= 0.604725417164
Epoch: 009 cost= 0.943649695979 accuracy= 0.619838230588
Epoch: 010 cost= 0.924821168184 accuracy= 0.629416770405
Epoch: 011 cost= 0.896670351426 accuracy= 0.648786714783
Epoch: 012 cost= 0.899215035968 accuracy= 0.64410387697
Epoch: 013 cost= 0.883694005233 accuracy= 0.641762455304
Epoch: 014 cost= 0.879168560108 accuracy= 0.647722437978
Epoch: 015 cost= 0.884021762345 accuracy= 0.642613875093
Epoch: 016 cost= 0.820838715191 accuracy= 0.681566629145
Epoch: 017 cost= 0.856995729385 accuracy= 0.659855257582
Epoch: 018 cost= 0.818622213823 accuracy= 

In [15]:
with tf.Session() as sess:
  saver.restore(sess, "./models/3layer-lr=0.0001.ckpt")
  print("Model restored.")
  
  _, loss_batch, logits_batch = sess.run([optimizer, loss, logits],
                                         feed_dict={X: X_tests, Y: y_test_oh})
  preds = tf.nn.softmax(logits_batch)
  correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(y_test_oh, 1))
  accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32))
  pred_acc = sess.run(accuracy)
  print ("Accuracy {0}".format(pred_acc / 522))

Model restored.
Accuracy 0.5632183908045977
